## Exercise 1

Move this code from `numpy` to `cupy`. Note that GPU operations are asynchronous!

Rembember to `cp.cuda.Device().synchronize()` when timing the execution.

In [ ]:
import numpy as np
import time

N = 100_000_000

# Create arrays on CPU
a = np.ones(N, dtype=np.float32)
b = np.ones(N, dtype=np.float32) * 2

# Perform vector addition
start = time.time()
c = a + b + np.sin(a)
end = time.time()

print("Sum of all elements:", np.sum(c))
print("Time (CPU):", end - start, "seconds")

In [ ]:
# do the same with cupy now

## Exercise 2

Let's play with copies now!

Fill the gaps for the creation and copies of the arrays. Then you can create the arrays directly on the GPU to avoid the host to device copy.

In [ ]:
import numpy as np
import cupy as cp
import time

N = 100_000_000 

# create two arrays of N random numbers on CPU
h_a = # TODO
h_b = # TODO

# copy it to GPU
start_htod = time.time()
d_a = # TODO
d_b = # TODO

# synchronize

end_htod = time.time()

print(f"Host to Device copy: {(end_htod - start_htod)*1e3:.2f} ms")

# perform some operation on the GPU
d_c = d_a + d_b

# copy back to the CPU
start_dtoh = time.time()
h_c_from_gpu = # TODO 

# synchronize

end_dtoh = time.time()
print(f"Device to Host copy: {(end_dtoh - start_dtoh)*1e3:.2f} ms")

# compare results
h_c = h_a + h_b
if np.allclose(h_c, h_c_from_gpu, rtol=1e-5):
    print("\nPassed :)")
else:
    print("\nFailed :(")

## Exercise 3

Take your favorite kernel and write it with cupy using `cp.RawKernel` or `cp.ElementwiseKernel` (or try filling an array, summing two arrays..).

Remember that with `cp.ElementwiseKernel` you can use the special variable `i` for the the index within the loop and method `_ind.size()` for the total number of elements to apply the elementwise operation.

In [ ]:
import cupy as cp
import numpy as np

In [ ]:
# Raw CUDA kernel
kernel_code = r"""
// kernel code
"""

mykernel = cp.RawKernel(kernel_code, "mykernel")

# Launch configuration
threads_per_block = 256
blocks_per_grid = # TODO

# Launch kernel
fill_indices_kernel((blocks_per_grid,), 
                    (threads_per_block,), 
                    ('''kernel arguments'''))

In [ ]:
# CuPy's elementwise kernel
fill_indices_kernel = cp.ElementwiseKernel(
    '',  # list of inputs 
    '',  # list of outputs
    '',  # operation
    ''   # kernel name
)

# Launch the elementwise kernel
fill_indices_kernel('''input and outputs''')

## Exercise 4

Reduction time!

NOTE: you can use `cp.dtype(cp.float32).itemsize` to obtain the size of a `float` in bytes.

In [ ]:
import cupy as cp
import numpy as np

N = 1024 
a = cp.arange(N, dtype=cp.float32)

block_size = 256
grid_size = # TODO
partial_sums = # TODO (an empty array of ? elements to store the partial sums on GPU)

# CUDA kernel: reduce a block of input into shared memory
kernel_code = r'''
// your kernel code
'''

reduce_kernel = cp.RawKernel(kernel_code, "reduce_sum")

# First reduction pass over the input array
shared_mem_size = # TODO (how much shared memory is needed?)
reduce_kernel((grid_size,), (block_size,),
              (a, partial_sums, np.int32(N)),
              shared_mem=shared_mem_size)

# Second reduction pass over partial_sums to get final result
# We can use the same kernel to reduce the partial sums array
partial_sums_out = # TODO (an empty array of ? elements to store the result on GPU)

grid_size = # TODO
reduce_kernel((grid_size,), (block_size,),
              (partial_sums, partial_sums_out, np.int32(grid_size)),
              shared_mem=shared_mem_size)

cp.cuda.Device().synchronize()  # ensure GPU work is complete

# Verify
expected = np.sum(cp.asnumpy(a))
computed = float(partial_sums_out[0])

if np.allclose(computed, expected):
    print("\nPassed!")
else:
    print("RawKernel reduction:", computed)
    print("Expected:           ", expected)

In [ ]:
import cupy as cp
import numpy as np

N = 1024
a = cp.arange(N, dtype=cp.float32)

# with reduction kernel
reduction_kernel = cp.ReductionKernel(
    '',      # input params
    '',      # output param
    '',      # map
    '',      # reduce
    '',      # post-reduction
    '',      # identity value
    ''       # kernel name
)
total = reduction_kernel(a)

# with built in python function
total2 = # TODO

expected = np.sum(cp.asnumpy(a))
if np.allclose(total, expected):
    print("\nReductionKernel passed!")
if np.allclose(total2, expected):
    print("\ncp.sum passed!")